<a href="https://colab.research.google.com/github/Ryunoshin3150/PLS-SEM_Ryunoshin/blob/main/simulations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
library(matrixpls)
library(psych)
library(simsem)
library(lavaan)
library(parallel)
library(MASS)
library(Matrix)
library(heplots)

MULTICORE <- FALSE
DEBUG <- FALSE
SAVERESULTS <- TRUE
REPLICATIONS <- 1000
ESTIMATESETS <- 1:2

source("parameters.R")

args <- commandArgs(trailingOnly = TRUE)
if (length(args) == 0) {
  designNumbers <- 1:nrow(designMatrix)
} else {
  designNumbers <- (as.numeric(args[1]))
}

for (designNumber in designNumbers) {
  set.seed(12345)
  design <- designMatrix[designNumber, ]
  print(paste("Starting design number", designNumber))
  print(design)

  MODEL <- MODELS[design$model]
  parTable <- lavaanify(MODEL)
  loadingsParTable <- parTable[parTable$op == "=~",]
  observedVariableNames <- unique(loadingsParTable$rhs)

  cvMat <- diag(length(observedVariableNames))
  colnames(cvMat) <- rownames(cvMat) <- observedVariableNames
  fit <- lavaan::lavaan(MODEL, sample.cov = cvMat, sample.nobs = 100)

  dataTemplate <- simsem::model.lavaan(fit)

  BE <- rawDraw(dataTemplate@dgen[[1]]$BE)$param * design$scalingFactor
  PS <- rawDraw(dataTemplate@dgen[[1]]$PS)$param
  i <- diag(nrow(PS)) == 0
  PS[i] <- PS[i] * design$scalingFactor
  diag(PS) <- 1 - diag(BE %*% PS %*% t(BE))

  dataTemplate@dgen[[1]]$PS@free <- apply(PS, 2, as.character)
  dataTemplate@dgen[[1]]$BE@free <- apply(BE, 2, as.character)

  Lambda <- rawDraw(dataTemplate@dgen[[1]]$LY)$param

  dist <- bindDist("norm", kurtosis = design$excessKurtosis)

  print("Generating datasets")
  dataSets <- sim(
    REPLICATIONS, dataTemplate, design$sample, dataOnly = TRUE,
    factDist = dist, errorDist = dist,
    sequential = TRUE, saveLatentVar = TRUE,
    multicore = MULTICORE
  )

  estimator <- design$estimator

  if (substr(estimator,1,3)=="CFA" || substr(estimator,1,3)=="SEM") {

    if (substr(estimator,1,3)=="SEM") {
      estimatedModel <- (gsub("f[0-9] ~~ f[0-9]","",gsub("[.0-9]+\\*","",gsub("\ni.*","",MODEL))))
      lavaanfun <- "sem"
    } else {
      estimatedModel <- gsub("f[0-9] ~ .*$","",gsub("[.0-9]+\\*","",gsub("\ni.*","",MODEL)))
      lavaanfun <- "cfa"
    }

    inner <- matrixpls:::parseModelToNativeFormat(MODEL)$inner
    exog <- which(rowSums(inner) == 0)

    sim.res <- sim(
      model = estimatedModel, rawData = dataSets, lavaanfun = lavaanfun,
      outfundata = function(fit,data) {
        var.lv <- diag(inspect(fit,"cov.lv"))
        var.ov <- unlist(lapply(data,function(x){
          sum((x-mean(x))^2)/length(x)
        }))
        sd.lv <- sqrt(var.lv)
        sd.ov <- sqrt(var.ov)

        C <- cov2cor(inspect(fit,"cov.lv"))
        S <- matrix(0,0,0,dimnames = list(c(),c()))

        regressions <- matrixpls:::estimatesMatrixToVector(
          estimator.regression(S, inner, W = NULL, C = C), inner, "~"
        )

        lambda <- inspect(fit,"coef")$lambda * ((1/sd.ov) %o% sd.lv)
        loadings <- lambda[lambda != 0]
        names(loadings) <- paste(colnames(lambda)[col(lambda)[lambda != 0]],
                                 "=~", rownames(lambda)[row(lambda)[lambda != 0]], sep="")

        exogC <- C[exog,exog]
        correlations <- exogC[lower.tri(exogC)]
        names(correlations) <- paste(colnames(exogC)[col(exogC)[lower.tri(exogC)]],
                                     "~~", rownames(exogC)[row(exogC)[lower.tri(exogC)]], sep="")

        inadmissible <- ifelse(any(eigen(C)$values < 0),1,0)

        c(inadmissible,regressions,loadings,correlations)
      },
      multicore = MULTICORE
    )

    sim.res@coef <- as.data.frame(do.call(rbind,lapply(sim.res@extraOut, function(x){
      if(is.null(x)) return(NA)
      x[-1]
    })))

    estimates <- sim.res@coef[grep("~f",names(sim.res@coef))]
    results <- cbind(inadmissible = unlist(lapply(sim.res@extraOut, function(x){
      if(is.null(x)) return(1)
      x[1]
    })), estimates)

    if(SAVERESULTS) save(results, file = paste("Paths",designNumber,estimator,1,".Rdata", sep="_"))
  }

  else {
    print(paste("Estimating. Estimator:", estimator))
    estimatorIndex <- (estimator == "PLSc") + 1 * grepl("EIV",estimator) + 1
    if(estimator == "EIVminres") estimatorIndex <- 3

    modeIndex <- (estimator == "PLSc") + 1

    sim.res <- matrixpls.sim(
      model = MODEL, rawData = dataSets,
      disattenuate = (estimator == "PLSc" || grepl("EIV",estimator)),
      innerEstimator = inner.factor,
      parametersReflective = switch(estimatorIndex,
                                    estimator.PLScLoadings,
                                    estimator.CFALoadings,
                                    estimator.EFALoadings),
      weightFunction = switch(modeIndex,
                              weight.fixed,
                              weight.pls),
      boot.R = FALSE, fitIndices = NULL,
      multicore = MULTICORE,
      stopOnError = DEBUG
    )

    results <- do.call(lapply, list(sim.res@extraOut, function(pls.res) {
      if(is.null(pls.res)) return(NA)
      R <- attr(pls.res,"R")
      names(R) <- paste("R",names(R))

      W <- attr(pls.res,"W")
      l <- loadings(pls.res)
      Q <- colSums(l * t(W))^2
      names(Q) <- paste("Q",names(Q))

      CR <- CR(pls.res)
      names(CR) <- paste("CR",names(CR))

      S <- attr(pls.res,"S")
      a <- apply(W,1,function(x){alpha(S[x!=0,x!=0])$total[1]})
      temp <- paste("Alpha",names(a))
      a <- unlist(a)
      names(a) <- temp

      c(trueR = R, Q, CR, a)
    }))
  }
}
